# Dynamic Pricing for Urban Parking Lots

This notebook implements three models:
1. Baseline Pricing
2. Demand-Based Pricing
3. Competitive Pricing (based on nearby lots)

It also includes real-time simulation using Pathway.

In [ ]:
!pip install bokeh pathway panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from math import radians, sin, cos, sqrt, atan2
import panel as pn
pn.extension('tabulator')
output_notebook()


In [ ]:
file_path = '/content/dataset.csv'
df = pd.read_csv(file_path)

print("Columns:", df.columns.tolist())
print(df.head())


Columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low         

In [ ]:
if 'timestamp' not in df.columns:
    if 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
        df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], errors='coerce')
    else:
        df['timestamp'] = pd.date_range(start='2024-01-01 08:00:00', periods=len(df), freq='30min')


In [ ]:
base_price = 10
alpha = 2.0
parking_lots = df['LotID'].unique() if 'LotID' in df.columns else df['SystemCodeNumber'].unique()
df['price'] = base_price

results = []
lot_column = 'LotID' if 'LotID' in df.columns else 'SystemCodeNumber'
for lot_id in parking_lots:
    lot_df = df[df[lot_column] == lot_id].sort_values('timestamp').copy()
    prices = [base_price]
    for i in range(1, len(lot_df)):
        capacity = lot_df.iloc[i-1]['Capacity']
        occ_ratio = lot_df.iloc[i-1]['Occupancy'] / capacity if capacity else 0
        prices.append(prices[-1] + alpha * occ_ratio)
    lot_df['price'] = prices
    results.append(lot_df)
baseline_df = pd.concat(results)


In [ ]:
plots = []
for lot_id in parking_lots[:5]:
    temp_df = baseline_df[baseline_df[lot_column] == lot_id]
    p = figure(title=f"Lot {lot_id} - Baseline Price", x_axis_type='datetime', width=700, height=300)
    p.line(temp_df['timestamp'], temp_df['price'], line_width=2)
    plots.append(p)
show(column(*plots))


In [ ]:
alpha = 2.0; beta = 1.5; gamma = 1.0; delta = 2.0; epsilon = 1.0; lambda_ = 0.5
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.3}
traffic_map = {'low': 1, 'average': 2, 'high': 3}
model2_results = []

for lot_id in parking_lots:
    lot_df = df[df[lot_column] == lot_id].sort_values('timestamp').copy()
    lot_df['occ_ratio'] = lot_df['Occupancy'] / lot_df['Capacity']
    lot_df['vehicle_weight'] = lot_df['VehicleType'].map(vehicle_weights).fillna(1.0)
    lot_df['traffic'] = lot_df['TrafficConditionNearby'].map(traffic_map).fillna(1)
    lot_df.fillna({'QueueLength': 0, 'IsSpecialDay': 0}, inplace=True)

    lot_df['demand_raw'] = (
        alpha * lot_df['occ_ratio'] +
        beta * lot_df['QueueLength'] -
        gamma * lot_df['traffic'] +
        delta * lot_df['IsSpecialDay'] +
        epsilon * lot_df['vehicle_weight']
    )
    lot_df['demand_norm'] = (lot_df['demand_raw'] - lot_df['demand_raw'].min()) / (lot_df['demand_raw'].max() - lot_df['demand_raw'].min() + 1e-6)
    lot_df['price'] = base_price * (1 + lambda_ * lot_df['demand_norm'])
    lot_df['price'] = lot_df['price'].clip(5, 20)
    model2_results.append(lot_df)
demand_df = pd.concat(model2_results)


In [ ]:
plots = []
for lot_id in parking_lots[:5]:
    temp_df = demand_df[demand_df[lot_column] == lot_id]
    p = figure(title=f"Lot {lot_id} - Demand-Based Price", x_axis_type='datetime', width=700, height=300)
    p.line(temp_df['timestamp'], temp_df['price'], color="orange", line_width=2)
    plots.append(p)
show(column(*plots))


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat, dlon = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

lot_coords = df.groupby(lot_column)[['Latitude', 'Longitude']].first()
model3_results = []

for ts in demand_df['timestamp'].unique():
    frame = demand_df[demand_df['timestamp'] == ts].copy()
    for _, row in frame.iterrows():
        current_lot = row[lot_column]
        lat1, lon1 = lot_coords.loc[current_lot]
        nearby = [ol for ol, coords in lot_coords.iterrows()
                  if ol != current_lot and haversine(lat1, lon1, coords['Latitude'], coords['Longitude']) < 0.5]
        avg_price = frame[frame[lot_column].isin(nearby)]['price'].mean() if nearby else row['price']
        price, occ_ratio = row['price'], row['Occupancy'] / row['Capacity']
        if occ_ratio > 0.9 and price > avg_price:
            price *= 0.95
        elif occ_ratio < 0.5 and price < avg_price:
            price *= 1.05
        row['price'] = min(max(price, 5), 20)
        model3_results.append(row)
comp_df = pd.DataFrame(model3_results)


In [ ]:
plots = []
for lot_id in parking_lots[:5]:
    temp_df = comp_df[comp_df[lot_column] == lot_id]
    p = figure(title=f"Lot {lot_id} - Competitive Price", x_axis_type='datetime', width=700, height=300)
    p.line(temp_df['timestamp'], temp_df['price'], color="green", line_width=2)
    plots.append(p)
show(column(*plots))


In [ ]:
!pip install pathway

In [ ]:
import pathway as pw
import pandas as pd
class ParkingSchema(pw.Schema):
    timestamp: pw.DateTimeNaive
    LotID: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic: float
    is_special_day: int
    vehicle_type: str

def run_pathway_stream_simulated(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.dropna(subset=['timestamp'])

    table = pw.debug.table_from_pandas(df, schema=ParkingSchema)
    table = table.select(
        timestamp=table.timestamp,
        LotID=table.LotID,
        price=10 + 2.0 * table.occupancy / table.capacity
    )
    pw.debug.compute_and_print(table)

rename_map = {
    'SystemCodeNumber': 'LotID',
    'Occupancy': 'occupancy',
    'Capacity': 'capacity',
    'QueueLength': 'queue_length',
    'TrafficConditionNearby': 'traffic_condition',
    'IsSpecialDay': 'is_special_day',
    'VehicleType': 'vehicle_type'
}
traffic_map = {'low': 1, 'average': 2, 'high': 3}

comp_df = comp_df.rename(columns=rename_map)
comp_df['traffic'] = comp_df['traffic_condition'].map(traffic_map).fillna(1)
comp_df['timestamp'] = pd.to_datetime(comp_df['timestamp'], errors='coerce')

final_df = comp_df[[
    'timestamp', 'LotID', 'occupancy', 'capacity',
    'queue_length', 'traffic', 'is_special_day', 'vehicle_type'
]].dropna()

run_pathway_stream_simulated(final_df.head(100))


            | timestamp           | LotID            | price
^5G57FE2... | 2016-01-11 08:06:00 | BHMBCCMKT01      | 10.173310225303293
^879Q0GF... | 2016-01-11 08:06:00 | BHMBCCTHL01      | 10.465116279069768
^5WWH5Y1... | 2016-01-11 08:06:00 | BHMEURBRD01      | 10.497872340425532
^PBQSJCW... | 2016-01-11 08:06:00 | BHMMBMMBX01      | 10.943231441048034
^JDB0W2F... | 2016-01-11 08:06:00 | BHMNCPHST01      | 10.438333333333333
^16C11MN... | 2016-01-11 08:06:00 | BHMNCPHST01      | 10.438333333333333
^FPXZG9H... | 2016-01-11 08:06:00 | BHMNCPNST01      | 10.7340206185567
^4579P4M... | 2016-01-11 08:06:00 | Broad Street     | 10.530434782608696
^KPPEQV3... | 2016-01-11 08:06:00 | Others-CCCPS105a | 10.612244897959183
^GN6FNHG... | 2016-01-11 08:06:00 | Others-CCCPS119a | 10.128433820906173
^1WXC7GY... | 2016-01-11 08:06:00 | Others-CCCPS135a | 10.598506309554468
^D43VR7R... | 2016-01-11 08:06:00 | Others-CCCPS202  | 10.307116104868914
^JZC283K... | 2016-01-11 08:06:00 | Others-CCCPS8    

In [ ]:
baseline_df.to_csv("baseline_model.csv", index=False)
demand_df.to_csv("demand_model.csv", index=False)
comp_df.to_csv("competitive_model.csv", index=False)

from google.colab import files
files.download("baseline_model.csv")
files.download("demand_model.csv")
files.download("competitive_model.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>